In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import mysql.connector
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [3]:
cnx = mysql.connector.connect(user='root', password='taskbase',
                              host='127.0.0.1', port='3309', database='lernnavi')

In [4]:
query = "SELECT * FROM DocumentTypes"
df_types = pd.read_sql(query, cnx)
df_types.rename(columns={'id':'typeId', 'name':'type'}, inplace=True)
df_types.head()

,typeId,type
0,26,BIT_MATCH
1,32,BIT_MATCH_MATRIX
2,27,BIT_MATCH_PICTURE
3,30,BIT_MULTIPLE_CHOICE
4,31,BIT_MULTIPLE_RESPONSE


In [5]:
query = "SELECT * FROM Documents"
df_tasks = pd.read_sql(query, cnx)
df_tasks = pd.merge(df_tasks, df_types, how="inner", on='typeId')
df_tasks.head()

,version,documentId,title,typeId,createdTime,authorId,content,Status,versionComment,aiStatus,feedbackComment,customerId,language,followUpTask,type
0,8,Dyx07vCUuWqWSiHC6V,,3,1531728113426,2,"{""id"": ""Dyx07vCUuWqWSiHC6V"", ""type"": ""HIGHLIGH...",WORK_IN_PROGRESS,None,INITIAL,None,None,None,None,HIGHLIGHT
1,9,Dyx07vCUuWqWSiHC6V,Aufgabe PT1.2,3,1531728228715,2,"{""id"": ""Dyx07vCUuWqWSiHC6V"", ""type"": ""HIGHLIGH...",WORK_IN_PROGRESS,None,INITIAL,None,None,None,None,HIGHLIGHT
2,20,HurIW8u0XJAWSiN5aR,,3,1531729909418,2,"{""id"": ""HurIW8u0XJAWSiN5aR"", ""type"": ""HIGHLIGH...",WORK_IN_PROGRESS,None,INITIAL,None,None,None,None,HIGHLIGHT
3,21,HurIW8u0XJAWSiN5aR,,3,1531730025414,2,"{""id"": ""HurIW8u0XJAWSiN5aR"", ""type"": ""HIGHLIGH...",WORK_IN_PROGRESS,None,INITIAL,None,None,None,None,HIGHLIGHT
4,24,HurIW8u0XJAWSiN5aR,Aufgabe PT1.6a,3,1531730573408,2,"{""id"": ""HurIW8u0XJAWSiN5aR"", ""type"": ""HIGHLIGH...",WORK_IN_PROGRESS,None,INITIAL,None,None,None,None,HIGHLIGHT


In [6]:
query = "SELECT * FROM LatestDocumentVersions"
df = pd.read_sql(query, cnx)
df_latest_task_versions = pd.merge(df_tasks, df, how="inner", on=["version", "documentId"])
df_latest_task_versions.head()

,version,documentId,title,typeId,createdTime_x,authorId,content,Status,versionComment,aiStatus,feedbackComment,customerId,language,followUpTask,type,id,createdTime_y
0,2325,PT1-11a-restored,Aufgabe PT1.11a,3,1535643328825,2,"{""id"": ""PT1-11a-restored"", ""type"": ""HIGHLIGHT""...",WORK_IN_PROGRESS,None,INITIAL,None,None,None,None,HIGHLIGHT,2693,1534434704301
1,2388,7kQh-veLQfOanjx1QkhbKx,Aufgabe PT 2.14a,3,1535660365797,2,"{""id"": ""7kQh-veLQfOanjx1QkhbKx"", ""type"": ""HIGH...",WORK_IN_PROGRESS,None,INITIAL,None,None,None,None,HIGHLIGHT,1589,1532335779747
2,2395,d7SwsoVyAUP80fMioyhxzH,Aufgabe PT 2.14c,3,1535661392063,2,"{""id"": ""d7SwsoVyAUP80fMioyhxzH"", ""type"": ""HIGH...",WORK_IN_PROGRESS,None,INITIAL,None,None,None,None,HIGHLIGHT,2516,1532336772750
3,3786,4R58yTMxw8b5CK1F72eh0W,Musteraufgabe D_Markieren_Adjektive_3.4_WORT_A_1,3,1562516756370,32,"{""id"": ""4R58yTMxw8b5CK1F72eh0W"", ""type"": ""HIGH...",WORK_IN_PROGRESS,None,INITIAL,None,None,None,None,HIGHLIGHT,927,1559076209311
4,3788,2UiRhzBKL217r4pFiCA6Ek,Ersatz 1.6b_Markieren mit mehreren Farben,3,1562516867617,32,"{""id"": ""2UiRhzBKL217r4pFiCA6Ek"", ""type"": ""HIGH...",WORK_IN_PROGRESS,None,INITIAL,None,None,None,None,HIGHLIGHT,425,1555145282057


In [7]:
columns_to_keep = ["documentId", "type", "title", "language"]
df = df_latest_task_versions[columns_to_keep]

In [8]:
df_content = df_latest_task_versions["content"].apply(json.loads).apply(pd.Series)
df_content.rename(columns={"id":"documentId"}, inplace=True)

In [9]:
df_open_tasks = df_content[df_content["type"] == "OPEN"].dropna(axis=1, how="all")
columns_to_keep = ["documentId", "language", "title", "description", "context", "topic", "hints", "solutionSteps", "replacements", "resource", "sampleSolutions", "stepifier"]
df_open_tasks = df_open_tasks[columns_to_keep]
# df_open_tasks.head()

**Cleaning description**

In [10]:
df_open_tasks["description"] = df_open_tasks["description"].apply(lambda str: BeautifulSoup(str, 'html.parser').get_text().strip())
df_open_tasks.head()

,documentId,language,title,description,context,topic,hints,solutionSteps,replacements,resource,sampleSolutions,stepifier
13847,4HNL62l.47Ta0sHqrOzKoF,NaN,Fragebogen SG 10,Was hat dir besonders gut gefallen?,NaN,NaN,[],[],NaN,NaN,[],NaN
13848,aCF0hfb1kQEayDOqwEp0s1,NaN,Fragebogen SG 11,Was könnte verbessert werden?,NaN,NaN,[],[],NaN,NaN,[],NaN
13849,dCZ4cWixQrM9ovYwc-TP.z,NaN,Fragebogen SG 13,Welche wichtigen Features fehlen in unserer Pl...,NaN,NaN,[],[],NaN,NaN,[],NaN
13850,8iXTTKYbQ.Hbd8daUV8dcQ,NaN,PH nach 2,Was hat Ihnen besonders gut gefallen?,NaN,NaN,[],[],NaN,NaN,[],NaN
13851,eEFZ3WW6kHx9rb9tdtx0AZ,NaN,PH nach 3,Was könnte verbessert werden?,NaN,NaN,[],[],NaN,NaN,[],NaN


**Fixing language field**

In [11]:
from langdetect import detect

In [12]:
existing_language = df_open_tasks.dropna(subset=["language"])
missing_language = df_open_tasks[df_open_tasks['language'].isna()]

In [13]:
len(df_open_tasks) == len(existing_language) + len(missing_language)

True

In [14]:
def detect_lang(text):
    try:
        lang = detect(text)
        return lang.upper()
    except:
        return ""
        
missing_language["language"] = missing_language["description"].apply(lambda text: detect_lang(text))

In [15]:
missing_language["language"].unique()

array(['DE', '', 'EN', 'SO', 'FR', 'IT', 'NL', 'NO', 'ES', 'ET', 'TR',
       'AF', 'UK', 'SV', 'DA'], dtype=object)

In [16]:
L = len(missing_language)
langs = ["DE", "EN", "FR", "IT"]
l = len(missing_language[missing_language["language"].isin(langs)])
print("Percentage of descriptions that were not correctly detected {:.2f}%".format((L - l) / L ))

Percentage of descriptions that were not correctly detected 0.09%


In [17]:
# Remove languages that are not DE, EN, FR or IT
missing_language = missing_language[missing_language["language"].isin(langs)]
df_open_tasks = pd.concat([existing_language, missing_language], ignore_index=True)

In [18]:
df_open_tasks["word_count"] = df_open_tasks["description"].apply(lambda s: len(s.split()))
df_open_tasks = df_open_tasks[(df_open_tasks["word_count"] > 2) & (df_open_tasks["word_count"] < 200)]

In [20]:
df_open_tasks.to_csv("data/open_tasks.csv", index_label=False)